In [3]:
import numpy as np
import pandas as pd

In [4]:
data_dir = '/home/vadim/playlist_generation/data'

In [5]:
tracks = pd.read_csv('{}/tracks.csv'.format(data_dir), index_col=0)

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
print ('Number of most nonpopular',tracks[tracks.popular == 1].shape[0], '\n', 
       'Popularity of most popular', tracks.popular.max(), '\n',
       'Mean popularity', tracks.popular.mean(), '\n',
       'Median',tracks.popular.median(), '\n',
       'Number of users>=10',tracks[tracks.popular >= 10].shape[0], '\n'
       'Number transactions', tracks.popular.sum())
tracks.head()

Number of most nonpopular 638612 
 Popularity of most popular 13977 
 Mean popularity 15.997936875138299 
 Median 1.0 
 Number of users>=10 166426 
Number transactions 19881836


,trackid,popular,artistid,albumid
0,774269,2,3810,76935
1,432553,2,7239,13835
2,1236511,1,133426,446132
3,969465,1,6838,53879
4,302436,6,16816,35620


In [7]:
artists = pd.read_csv('{}/artists.csv'.format(data_dir), index_col=0)

In [8]:
print ('Number of most nonpopular',artists[artists.popular == 1].shape[0], '\n',
       'Popularity of most popular', artists.popular.max(), '\n'
       'Mean popularity', artists.popular.mean(), '\n',
       'Median',artists.popular.median(), '\n',
       'Number of users>=10',artists[artists.popular >= 10].shape[0], '\n',
       'Number transactions', artists.popular.sum())
artists.head()

Number of most nonpopular 65530 
 Popularity of most popular 251773 
Mean popularity 111.15181331797775 
 Median 3.0 
 Number of users>=10 47295 
 Number transactions 19881836


,artistid,popular
0,58476,1
1,133660,1
2,42343,3
3,103158,1
4,1656,3577


In [9]:
albums = pd.read_csv('{}/albums.csv'.format(data_dir), index_col=0)
print ('Number of the most nonpopular',albums[albums.popular == 1].shape[0], '\n',
       'Popularity of the most popular', albums.popular.max(), '\n',
       'Mean popularity', albums.popular.mean(), '\n',
       'Median',albums.popular.median(), '\n',
       'Number of users>=10',albums[albums.popular >= 10].shape[0], '\n',
       'Number transactions', albums.popular.sum())
albums.head()

Number of the most nonpopular 166217 
 Popularity of the most popular 46526 
 Mean popularity 44.37731519715726 
 Median 2.0 
 Number of users>=10 106367 
 Number transactions 19881836


,albumid,popular
0,86118,1
1,362627,3
2,104733,67
3,266531,4
4,60761,46


In [10]:
transactions = pd.read_csv('{}/transactions.csv'.format(data_dir), index_col=0)

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
transactions.head()

,pid,trackid
0,822000,0
1,822000,1
2,822000,2
3,822000,3
4,822000,4


In [34]:
playlists = pd.read_csv('{}/playlists.csv'.format(data_dir), index_col=0)
playlists.head()

,pid,name,num_followers
0,822000,rap,1
1,822001,wedding mix,1
2,822002,Party,1
3,822003,wElP,1
4,822004,Worship,1


In [35]:
playlists.shape

(300000, 3)

In [36]:
playlists.name.unique().shape

(52531,)

In [37]:
playlists['name'].values[2]

'Party'

In [38]:
playlists['name'] = [str(playlists['name'].values[i]).lower() for i in range(playlists.shape[0])]

In [39]:
playlists.name.unique().shape

(29569,)

In [40]:
grouped = playlists[['pid', 'name']].groupby('name')

In [41]:
grouped.count().sort_values(by='pid', ascending=False).head(10)

,pid
name,
chill,2803
country,2738
workout,2424
rap,2412
oldies,2249
christmas,1962
rock,1924
party,1655
throwback,1542


In [42]:
tracks.popular.quantile(q=.5)

1.0

In [43]:
def get_threholds(tracks, parts=[1, 0]):
    threholds = []
    n = len(parts) - 1
    for i in range(n):
        t1 = round(tracks.popular.quantile(q=parts[i]))
        t2 = round(tracks.popular.quantile(q=parts[i+1]))
        if i == n - 1:
            t2 = 0
        threholds.append((t1, t2))
    return threholds
thrs = get_threholds(tracks, [1, 0.999, 0.98, 0.88, 0])
thrs

[(13977, 2039), (2039, 109), (109, 11), (11, 0)]

In [44]:
tracks.popular.unique().shape

(2839,)

In [102]:
s = 'the music rock on heart'

In [111]:
reg = re.compile('the ')
reg.sub('', s)

' music rock on heart'

In [45]:
names = playlists['name'].values
for i in range(names.shape[0]):
    names[i] = re.compile('[^a-zA-Z0-9 ]').sub('', names[i])

In [55]:
playlists['name'] = names
grouped = playlists[['pid', 'name']].groupby('name')
grouped.count().sort_values(by='pid', ascending=False).head(50)

,pid
name,
,4605
chill,2895
country,2859
workout,2482
rap,2475
oldies,2282
christmas,2236
rock,1988
party,1751


In [87]:
words_in_names = []
for i in range(names.shape[0]):
    words_in_names += names[i].split(' ')

In [88]:
words_in_names = pd.DataFrame({'word': words_in_names, 'id':np.arange(len(words_in_names))})

In [89]:
words_in_names

,id,word
0,0,rap
1,1,wedding
2,2,mix
3,3,party
4,4,welp
5,5,worship
6,6,sad
7,7,nibba
8,8,hours
9,9,country


In [90]:
grouped = words_in_names.groupby('word')

In [94]:
grouped.count().sort_values(by='id', ascending=False)[50:100]

,id
word,
hop,1437
run,1392
hip,1380
spring,1367
slow,1346
for,1329
classics,1297
trip,1288
running,1279
